In [30]:
import os
import codecs
import pickle

def save(filename, data):
    with open(filename, 'wb') as output:
        pickle.dump(data, output)

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

path = 'E:\Fudan\part-whole relations\data_test.txt'
filename = 'E:\Fudan\part-whole relations\entities_list.pkl'

entities = []
with codecs.open(path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f.readlines():
        tmp = ''
        cnt = 0
        entity = []
        for strs in line:
            tmp += strs
            if strs == '\t':
                cnt += 1
                if cnt <= 3:
                    tmp = ''
                else:
                    entity.append(tmp.replace('\t', ''))
                    tmp =''
                if cnt >= 6:
                    entities.append(entity)
                    break


entities_ = []
for term in entities:
    entities_.append(term[0])
    entities_.append(term[2])

save(filename, entities_)



In [16]:
import numpy as np
from sklearn.cluster import KMeans

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

filename1 = 'E:\Fudan\part-whole relations\embedding_dict.pkl'
filename2 = 'E:\Fudan\part-whole relations\entities_list.pkl'

embedding_dict = load(filename1)
entities = load(filename2)

entities_order = {}
offset_list = []
cnt = 0
for i in range(len(entities)) :
    if i % 2 == 0:
        entity1 = entities[i]
        entity2 = entities[i+1]
        entities_order[cnt] = zip(entity1, entity2)
        cnt += 1
        offset_list.append(embedding_dict[entity1] - embedding_dict[entity2])

X = np.array(offset_list)
kmeans = KMeans(n_cluster = k).fix(X)

labels = kmeans.labels_
clusters_cents = kmeans.clusters_centers_

clusters_dict = {}   # key是类别i，value是一个属于类别i的元素组成的列表
for i in range(len(labels)):
    if clusters_dict.get(labels[i]) == None:
        clusters_dict[labels[i]] = [i,]
    else:
        clusters_dict[labels[i]].append(i)


selected_list = []
for i in range(k):
    minDis = float('inf')  # 初始化为最大值
    flag = -1
    for j in cluster_dict[i]:
        # 计算类别i的形心与所属类别i的每个点的欧式距离
        distance = np.sqrt(np.sum(np.square(X[j] - clusters_cents[i])))
        if minDis > distance:
            minDis = distance
            flag = j
    selected_list.append(entities_order[flag])
    


    







In [29]:
import os
import codecs
import pickle

def save(filename, data):
    with open(filename, 'wb') as output:
        pickle.dump(data, output)
        
        
path = 'E:\Fudan\part-whole relations\glove.840B.300d.txt'
filename = 'E:\Fudan\part-whole relations\embedding_dict.pkl'

cnt = 0
dicts = {}
with codecs.open(path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f.readlines():
        lists = line.split()
        dicts[line[0]] = [float(x) for x in line[1:]]

save(filename, dicts)
        
            



{',': [-0.082752, 0.67204, -0.14987, -0.064983, 0.056491, 0.40228, 0.0027747, -0.3311, -0.30691, 2.0817, 0.031819, 0.013643, 0.30265, 0.0071297, -0.5819, -0.2774, -0.062254, 1.1451, -0.24232, 0.1235, -0.12243, 0.33152, -0.006162, -0.30541, -0.13057, -0.054601, 0.037083, -0.070552, 0.5893, -0.30385, 0.2898, -0.14653, -0.27052, 0.37161, 0.32031, -0.29125, 0.0052483, -0.13212, -0.052736, 0.087349, -0.26668, -0.16897, 0.015162, -0.0083746, -0.14871, 0.23413, -0.20719, -0.091386, 0.40075, -0.17223, 0.18145, 0.37586, -0.28682, 0.37289, -0.16185, 0.18008, 0.3032, -0.13216, 0.18352, 0.095759, 0.094916, 0.008289, 0.11761, 0.34046, 0.03677, -0.29077, 0.058303, -0.027814, 0.082941, 0.1862, -0.031494, 0.27985, -0.074412, -0.13762, -0.21866, 0.18138, 0.040855, -0.113, 0.24107, 0.3657, -0.27525, -0.05684, 0.34872, 0.011884, 0.14517, -0.71395, 0.48497, 0.14807, 0.62287, 0.20599, 0.58379, -0.13438, 0.40207, 0.18311, 0.28021, -0.42349, -0.25626, 0.17715, -0.54095, 0.16596, -0.036058, 0.08499, -0.64989,